In [1]:
from tardis.atomic import AtomData
import numpy as np
import pandas as pd

atomic_dataset = AtomData.from_hdf5('kurucz_cd23_chianti_H_He.h5')

tardis.atomic - INFO - Read Atom Data with UUID=5ca3035ca8b311e3bb684437e69d75d7 and MD5=21095dd25faa1683f4c90c911a00c3f8


In [2]:
def get_index(ele):
    index=-1
    for atomic_no,row in atomic_dataset.atom_data.iterrows():
        if ele in row['name']:
            index = atomic_no
            break
    return index

In [3]:
def parse_file(fname):
    with open(fname,'r') as f:
        for line in f:
            items = line.split()
            n = len(items)
            
            if 'data points' in line:
                df = pd.DataFrame(columns=np.arange(int(items[n-1])),
                         index=pd.Index([],
                                        name='element'),
                         dtype=np.float64)            
            
            if 'mass fraction\n' in line:
                    abundances = []
                    element_string = items[0]
                    atomic_no = get_index(element_string.capitalize())
                    element_symbol = atomic_dataset.atom_data.loc[atomic_no]['symbol']
                    
                    #Its a Isotope
                    if n==4:
                        element_symbol+=items[1]
                    
                    for line in f:
                        items =line.split()
                        if items:
                            abundances.extend( np.array(items).astype(np.float64))
                        else:
                            break
                    
                    df.loc[element_symbol] = abundances

    return df.transpose()
                

In [4]:
df = parse_file('DDC15_SN_HYDRO_DATA_0.976d')
df.to_csv('test.csv',index=False,sep=' ')